In [1]:
import pickle
import re

demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/UKB/'

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

print(list(demo.keys()))

['f_22001_0_0', 'f_21003_3_0', 'f_21003_2_0', 'f_21000_0_0', 'CaAfCarDiff', 'CaIndDiff', 'CaChiDiff', 'f_21000_0_0_comb', 'longitudinal', '22438-0.0', '22828-0.0', '26201-0.0', '26202-0.0', '26203-0.0', '26204-0.0', '26205-0.0', '26206-0.0', '26207-0.0', '26208-0.0', '26209-0.0', '26210-0.0', '26211-0.0', '26212-0.0', '26213-0.0', '26214-0.0', '26215-0.0', '26216-0.0', '26217-0.0', '26218-0.0', '26219-0.0', '26220-0.0', '26221-0.0', '26222-0.0', '26223-0.0', '26224-0.0', '26225-0.0', '26226-0.0', '26227-0.0', '26228-0.0', '26229-0.0', '26231-0.0', '26232-0.0', '26233-0.0', '26234-0.0', '26235-0.0', '26236-0.0', '26237-0.0', '26238-0.0', '26239-0.0', '26240-0.0', '26241-0.0', '26242-0.0', '26243-0.0', '26244-0.0', '26245-0.0', '26246-0.0', '26247-0.0', '26248-0.0', '26249-0.0', '26250-0.0', '26251-0.0', '26252-0.0', '26253-0.0', '26254-0.0', '26255-0.0', '26256-0.0', '26257-0.0', '26258-0.0', '26259-0.0', '26260-0.0', '26261-0.0', '26262-0.0', '26263-0.0', '26264-0.0', '26265-0.0', '262

In [24]:
import numpy as np

diff = []
field = '26203-0.0'
vals = []

for sub in demo['longitudinal']:
    try:
        p0 = np.load(f'{demodir}/fc/{sub}_task-20227-2_fc.npy')
        p1 = np.load(f'{demodir}/fc/{sub}_task-20227-3_fc.npy')
        v = demo[field][sub]
        vals.append(v)
        diff.append(p1-p0)
    except:
        continue
    
vals = np.array(vals)
diff = np.stack(diff)

print(len(vals), len(diff))

424 424


In [3]:
ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [7]:
bounds = [0, 30, 35, 49, 62, 120, 125, 156, 181, 199, 212, 221, 232, 236, 264]

def get_net_avg(bounds, net1, net2, diff):
    res = []
    
    s1 = np.arange(bounds[net1], bounds[net1+1])
    s2 = np.arange(bounds[net2], bounds[net2+1])

    for i,v in enumerate(diff):
        m = remap(vec2mat(v))
        c = m[np.ix_(s1,s2)].flatten()
        c = np.mean(c, keepdims=True)
        res.append(c)

    res = np.concatenate(res)
    
    return res

avg = get_net_avg(bounds, 0, 6, diff)
print(avg.shape)

(2652,)


In [25]:
import scipy.stats as stats

def corr(a, b, bonf=False):
    # Calculate correlation
    sigma_fv = np.einsum('a,a->',a,b)
    sigma_ff = np.einsum('a,a->',a,a)
    sigma_vv = np.einsum('a,a->',b,b)
    rho = sigma_fv/(sigma_ff*sigma_vv)**0.5
    # Sometimes happens with SNPs
#     rho[np.isnan(rho)] = 0
    # Get t distribution
    n = a.shape[0]
#     m = feats.shape[1]
    df = n-2
    #rho[rho > 0] = 0
    t = rho*(df/(1-rho**2))**0.5
    if t < 0:
        t = -t
#     t[t < 0] = -t[t < 0]
    # Convert to 2-sided p value
    p = (1-stats.t.cdf(t, df))*2
    # Bonferroni correction
    if bonf:
        p *= m
#     p[p > 1] = 1
#     p[p < 1e-5] = 1e-5
    return rho, p #np.log10(p)

for i in range(0,14):
    for j in range(i,14):
        avg = get_net_avg(bounds, i, j, diff)
        rho, p = corr(vals, avg)
        print(i,j,rho,p)

0 0 -0.04815814448476352 0.3225244321279441
0 1 -0.050552722002933814 0.299020861281059
0 2 -0.07415311090487665 0.12738438981465183
0 3 -0.026783649297450276 0.5823333339132795
0 4 -0.07022642239830937 0.14886155340498997
0 5 -0.08835899784686918 0.06912400069984392
0 6 -0.003531310784461433 0.942204240525391
0 7 -0.08913309744195028 0.06671404013687199
0 8 -0.11918435547686035 0.014062226229372587
0 9 -0.08651813492455969 0.07514203872796221
0 10 -0.028227191547927882 0.562162076481052
0 11 -0.0314127053690136 0.5188773583737505
0 12 -0.08839182665092439 0.06902036914007836
0 13 -0.059705206488170744 0.21987115171922555
1 1 -0.04011945264983716 0.4099377913821023
1 2 -0.0895167337191639 0.0655454865918319
1 3 -0.04717808938281858 0.332481237535597
1 4 -0.06598026705823606 0.17507240666342838
1 5 -0.0976811595215844 0.044404739345358823
1 6 -0.0044490904300904495 0.9272205440731485
1 7 -0.05867931871269614 0.22791262163446424
1 8 -0.10655331402947907 0.028246996759093035
1 9 -0.090471